In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'
sys.path.append(HOME + 'collaboration/')

In [3]:
import pandas as pd
import json
import numpy as np
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler

from math import log2
from utils import entropy

/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,
/usr/lib/python3/dist

In [5]:
# Page history

In [4]:
template='peacock'

In [5]:
with open(HOME+'features/activity_{}.json'.format(template),'rb') as f:
    features_pos = json.load(f)
with open(HOME+'negative_features/activity_{}.json'.format(template),'rb') as f:
    features_neg = json.load(f)

In [6]:
len(features_pos)

5174

In [7]:
len(features_neg)

4282

In [8]:
article_features_neg = pd.DataFrame([d['article'] for d in features_neg])
article_features_neg['label'] = 0
article_features_pos = pd.DataFrame([d['article'] for d in features_pos])
article_features_pos['label'] = 1

In [9]:
article_data = pd.concat([article_features_neg, article_features_pos])
labels = article_data['label']
feat = article_data.drop('label',axis=1)

In [10]:
len(article_data)

9456

In [11]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
8,frac_recent_revisions,0.346634,0.346634
11,concentration_ratio,0.152984,0.152984
13,recent_edit_size,0.116431,0.116431
0,edit_size,0.108846,0.108846
14,recent_response_time,0.103073,0.103073
12,contribution_frac_entropy,-0.100749,0.100749
9,top_contributor_frac,0.079093,0.079093
1,time_to_respond,0.078141,0.078141
10,frac_anon_revisions,-0.056411,0.056411
6,num_revisions,-0.052714,0.052714


In [12]:
# linear model

In [13]:
hypers = {'solver':'lbfgs','max_iter':10000,'C':10000}

In [14]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [15]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.7366428996228132

In [16]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[781, 272],
       [525, 786]])

In [17]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6635711270578303

In [18]:
# user-article

In [19]:
def aggregate_features(article_users):
    if len(article_users) == 0:
        return {}
    else:
        article_users = pd.DataFrame(article_users).drop('event_user_id',axis=1)
        means = {'mean_'+key:val for key, val in article_users.mean().items()}
        std = {'std_'+key:val for key, val in article_users.std().items()}
        max_ = {'max_'+key:val for key, val in article_users.max().items()}
        ent = {'frac_page_edits_ent': entropy(article_users.frac_page_edits)}
        features = {**means, **std, **max_, **ent}

        return features

In [20]:
user_article_features_pos = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_pos])
user_article_features_neg = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_neg])

In [21]:
user_article_features_pos['label'] = 1
user_article_features_neg['label'] = 0

user_article_data = pd.concat([user_article_features_neg,user_article_features_pos])
labels = user_article_data['label']
feat = user_article_data.drop('label',axis=1)

In [22]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
5,mean_frac_page_edits,0.233273,0.233273
24,frac_page_edits_ent,-0.120467,0.120467
0,mean_edit_size,0.096732,0.096732
21,max_frac_page_edits,0.090276,0.090276
13,std_frac_page_edits,0.082525,0.082525
2,mean_time_to_respond,0.074941,0.074941
3,mean_time_responded_to,0.069068,0.069068
20,max_num_edits,-0.058767,0.058767
12,std_num_edits,-0.058611,0.058611
10,std_time_to_respond,0.052783,0.052783


In [23]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.6557479861910241

In [25]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[638, 462],
       [499, 765]])

In [26]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6142111601766359

In [27]:
# talk vol

In [28]:
with open(HOME+'features/talk_{}.json'.format(template),'rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_{}.json'.format(template),'rb') as f:
    talk_neg = json.load(f)

In [29]:
talk_vol_neg = pd.DataFrame([d['talk_volume'] for d in talk_neg])
talk_vol_neg['label'] = 0
talk_vol_pos = pd.DataFrame([d['talk_volume'] for d in talk_pos])
talk_vol_pos['label'] = 1

In [30]:
talk_vol = pd.concat([talk_vol_neg,talk_vol_pos])
labels = talk_vol['label']
feat = talk_vol.drop('label',axis=1)

In [31]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
5,mean_response_time,0.062737,0.062737
6,page_talk_ratio,-0.033946,0.033946
1,frac_recent_revisions,0.030090,0.030090
2,num_editors,-0.027779,0.027779
0,num_revisions,-0.024701,0.024701
3,top_contributor_frac,0.017444,0.017444
4,mean_edit_size,-0.009398,0.009398


In [32]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [33]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.5714721447700177

In [34]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 257,  789],
       [ 230, 1088]])

In [35]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6810641627543035

In [38]:
# talk lang

In [36]:
with open(HOME+'features/talk_{}.json'.format(template),'rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_{}.json'.format(template),'rb') as f:
    talk_neg = json.load(f)

In [37]:
def aggregate_language(talk_feat):
    if len(talk_feat) == 0:
        return {}
    else:
        all_utts = pd.DataFrame(list(np.concatenate(list(talk_feat.values()))))
        means = {'mean_'+key:val for key, val in all_utts.mean().items()}
        std = {'std_'+key:val for key, val in all_utts.std().items()}
        max_ = {'max_'+key:val for key, val in all_utts.max().items()}
        features = {**means, **std, **max_}

        return features

In [38]:
talk_lang_neg = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_neg])
talk_lang_neg['label'] = 0

In [39]:
len(talk_lang_neg)

4282

In [40]:
talk_lang_pos = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_pos])
talk_lang_pos['label'] = 1

In [41]:
len(talk_lang_pos)

5174

In [42]:
talk_lang = pd.concat([talk_lang_neg,talk_lang_pos])
labels = talk_lang['label']
feat = talk_lang.drop('label',axis=1)

In [43]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
44,max_politeness_markers_==Please==,-0.029337,0.029337
45,max_politeness_markers_==Please_start==,-0.024005,0.024005
58,max_politeness_markers_==Indirect_(greeting)==,-0.023797,0.023797
55,max_politeness_markers_==1st_person_start==,-0.021296,0.021296
12,mean_politeness_markers_==2nd_person==,-0.020974,0.020974
65,max_reply_depth,-0.020399,0.020399
56,max_politeness_markers_==2nd_person==,-0.020345,0.020345
51,max_politeness_markers_==Gratitude==,-0.020281,0.020281
54,max_politeness_markers_==1st_person==,-0.019901,0.019901
10,mean_politeness_markers_==1st_person==,-0.018795,0.018795


In [44]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [55]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.7417919120374664

In [56]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[727, 325],
       [472, 840]])

In [57]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6782398062171983

In [170]:
# combined

In [58]:
labels = article_data['label']

feat = pd.concat([ds.reset_index(drop=True).drop('label',axis=1) 
                  for ds in [article_data,talk_vol,talk_lang,user_article_data]],axis=1) #editor_data,collab_data

In [59]:
len(feat)

9456

In [60]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
12,contribution_frac_entropy,0.088496,0.088496
2,article_age_years,0.076672,0.076672
109,max_frac_page_edits,-0.073533,0.073533
112,frac_page_edits_ent,0.070744,0.070744
9,top_contributor_frac,-0.070540,0.070540
10,frac_anon_revisions,0.070309,0.070309
101,std_frac_page_edits,-0.068979,0.068979
16,frac_recent_revisions,-0.065640,0.065640
93,mean_frac_page_edits,-0.061991,0.061991
17,num_editors,0.058880,0.058880


In [61]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)#,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [62]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.7398213612571471

In [63]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[756, 342],
       [478, 788]])

In [64]:
f1_score(y_pred=y_pred, y_true=y_test)

0.657762938230384

In [178]:
# Thoughts:

# article:
# comparing to same pages means some features are less relevant; eg. num_editors, mean edit_size, top_contributor_frac, etc
# recency features should be more important 
#     - calculate all features for recent revisions only as well?


# user-article
# empty for some negatives? sample page': 189559,'date': '2003-02-28 16:09:01'
# revert features are incorrect, need to re-export

In [ ]:
# Editor